In [345]:
import requests
from requests.exceptions import RequestException
from lxml import etree
import threading
import random
import json
from time import sleep
from time import strftime
from csv import DictWriter

In [176]:
dpath = './eu/'
pl = []
p = []

In [38]:
headers = {
	'X-Requested-With':'XMLHttpRequest'
}

In [78]:
s = 0
l = 10
while l == 10:
    print(s)
    url = f"https://drop.euphresco.net/search?from={s}"
    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            tmp = r.json()['data']
            pl.extend(tmp)
            l = len(tmp)
            s += 10
    except RequestException as err:
        print(err)
        print("Fail")

# 43.6s for 34 times

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340


In [79]:
len(pl)

347

In [81]:
with open(dpath+'pl.json', 'w') as out:
    json.dump(pl, out)
out.close()

In [327]:
def check(tmp):
    if len(tmp) == 0:
        return ''
    else:
        return tmp[0]

In [334]:
def checkStrip(tmp):
    if len(tmp) == 0:
        return ''
    else:
        return tmp[0].xpath('string(.)').strip()

In [336]:
def getP(id):
    sleep(random.randint(1,5))
    url = f"https://drop.euphresco.net/data/{id}"
    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            html = etree.HTML(r.content)
            g = html.xpath('//div[@class="widget addinfos"]//li/text()')
            p.append({
                'Title': html.xpath('//h2/text()')[0],
                'Type': html.xpath('//div[@class="col-md-4"]')[0].xpath('string(.)').strip(),
                'DOI': check(html.xpath('//small[count(@*)=0]/a/text()')),
                'Link': html.xpath('//td/a/@href')[0],
                'Authors': ("; ").join(html.xpath('//div[@class="margin10"]/p/a/text()')),
                'Description': checkStrip(html.xpath('//div[@class="margin10"]/div[@class="marginleft30"]')),
                'License': checkStrip(html.xpath('//a[@target="new"]/parent::div[@class="marginleft30"]')),
                'Funding': check(html.xpath('//span[contains(text(),"Funding")]/ancestor::div[@class="margin10 funding"]/div[@class="marginleft30"]/text()')),
                'Organisms': (",").join(html.xpath('//ul[@class="marginleft30"]/li/text()')),
                'Visible for': g[0].strip(),
                'Publication': g[1].strip(),
                'Last Update': g[2].strip(),
                'Published by': html.xpath('//div[@class="widget addinfos"]//li/a/text()')[0],
                'Categories': (", ").join(html.xpath('//div[@class="widget categories"]/a/text()')),
                'Tags': (", ").join(html.xpath('//div[@class="widget"]/a/text()'))
            })
    except RequestException as err:
        print(err)
        print("Fail")

In [193]:
len(p)

345

In [354]:
with open(dpath+'p.json', 'r') as f:
    pl = json.load(f)
len(pl)

347

In [353]:
def threadP():
	with open(dpath+'pl.json', 'r') as f:
		pl = json.load(f)
	threads = []
	for i in pl:
		t = threading.Thread(target=getP, args=(i['iddata'],))
		threads.append(t)
	for task in threads:
		task.start()
	for task in threads:
		task.join()
	with open(dpath+'p.json', 'w') as out:
		json.dump(p, out)
	out.close()

In [338]:
p = []
threadP()

In [355]:
def genCSV():
	with open(dpath+'p.json', 'r') as f:
		pdata = json.load(f)
	f.close()
	with open(dpath+'eu'+strftime("%Y%m%d-%H%M%S")+'.csv', 'w+', newline='',encoding="utf-8-sig") as out:
		writer = DictWriter(out, fieldnames=pdata[0].keys())
		writer.writeheader()
		writer.writerows(pdata)

In [351]:
genCSV()

In [337]:
html = getP('ee05f66a-e099-4f8b-b7dd-ea535426f037')

In [282]:
html.xpath('//h2/text()')[0]
#title

'An International Plant Sentinel Network as an early-warning system; research on future pest threats (IPSN II)'

In [283]:
html.xpath('//div[@class="col-md-4"]')[0].xpath('string(.)').strip()
# type

'Text'

In [288]:
html.xpath('//small[not(@*)]')

[<Element small at 0x1981b19e500>]

In [293]:
html.xpath('//small[count(@*)=0]/a/text()')[0]
# doi

'10.5281/zenodo.3908116'

In [296]:
html.xpath('//td/a/@href')[0]
# link

'https://drop.euphresco.net/getfile/6a703d59-a106-4945-b8f1-f0d501b57dbc'

In [245]:
("; ").join(html.xpath('//div[@class="margin10"]/p/a/text()'))
# author

'Ravnikar, Maja; Kutnjak Denis; Giovani, Baldissera; Fox, Adrian; Botermans, Marleen; Oplaat, Carla; Mollov, Dimitre'

In [301]:
html.xpath('//div[@class="margin10"]/div[@class="marginleft30"]')

[<Element div at 0x19828bedd00>]

In [303]:
tmp = html.xpath('//div[@class="margin10"]/div[@class="marginleft30"]')[0]
tmp.xpath('string(.)').strip()
# description

'The overall goal of the project ‘An International Plant Sentinel Network as an early-warning system; research on future pest threats (IPSN II)’ was to develop early warning systems for regulated and emerging plant pests and to provide information that can help prevent their introduction and mitigate their impact.The project focused on three main objectives:Enhancing early detection of regulated and emerging pestsSupporting of Pest Risk AnalysisEstablishing a self-sustainable network of botanic gardens and arboreta to implement the above objectives after the end of IPSN II.Botanic Gardens Conservation International (BGCI) coordinated the IPSN II project with funding from Department for Environment, Food and Rural Affairs (Defra) and scientific expertise from Fera Science Ltd. The Euphresco partners included experts from 16 institutions in Europe, Australasia and America representing botanic gardens and arboreta, universities, research institutes and National Plant Protection Organisati

In [304]:
html.xpath('//a[@target="new"]/parent::div[@class="marginleft30"]')

[<Element div at 0x1981b546000>]

In [305]:
tmp = html.xpath('//a[@target="new"]/parent::div[@class="marginleft30"]')[0]
tmp.xpath('string(.)').strip()
# license

'CC BY 4.0 - https://creativecommons.org/licenses/by/4.0/'

In [309]:
html.xpath('//span/text()')

['CC BY 4.0 - ', 'Euphresco']

In [320]:
html.xpath('//span[contains(text(),"Funding")]/ancestor::div[@class="margin10 funding"]/div[@class="marginleft30"]/text()')

['Euphresco']

In [306]:
html.xpath('//span[contains(text(),"Funding")]/ancestor::div[@class="margin10 funding"]/div[@class="marginleft30"]/text()')[0]
# funding

'Euphresco'

In [192]:
(", ").join([i.strip() for i in html.xpath('//ul[@class="marginleft30"]/li/text()')])

"'Candidatus Phytoplasma fragariae', Xanthomonas fragariae"

In [189]:
(",").join(html.xpath('//ul[@class="marginleft30"]/li/text()'))
# organisms

" 'Candidatus Phytoplasma fragariae' , Xanthomonas fragariae "

In [166]:
html.xpath('//div[@class="widget addinfos"]//li/text()')[3].strip()

''

In [135]:
html.xpath('//div[@class="widget addinfos"]//li/text()')
# publication date
# Last update

[' Public', ' 2020-06-25 00:00:00', ' 2023-01-01 20:02:36', ' ']

In [167]:
html.xpath('//div[@class="widget addinfos"]//li/a/text()')[0]
# published by

'EUPHRESCO Secretariat'

In [170]:
(", ").join(html.xpath('//div[@class="widget categories"]/a/text()'))
# categories

'Monitoring, Pest diagnostics, Surveillance'

In [171]:
(", ").join(html.xpath('//div[@class="widget"]/a/text()'))
# tags

'distribution, Euphresco project ID card, mapping, pcr, real-time pcr'